In [1]:
# clone the BLT repository
!git clone https://github.com/sathishkumar67/Byte-Latent-Transformer.git
# move the files to the current directory
!mv /kaggle/working/Byte-Latent-Transformer/* /kaggle/working/
# upgrade pip
!pip install --upgrade pip
# install latest version pytorch
# install the required packages
!pip install -r requirements.txt

Cloning into 'Byte-Latent-Transformer'...
remote: Enumerating objects: 162, done.
remote: Counting objects: 100% (162/162), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 162 (delta 91), reused 107 (delta 43), pack-reused 0 (from 0)
Receiving objects: 100% (162/162), 51.19 KiB | 5.69 MiB/s, done.
Resolving deltas: 100% (91/91), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 41.2 MB/s eta 0:00:0000:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 824.2/824.2 kB 26.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 47.8 MB/s  0:00:06m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.0 MB/s  0:00:00m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.6 MB/s  0:00:00m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.7 MB/s

In [2]:
# import os
# import shutil

# # Get current working directory
# current_dir = os.getcwd()

# for item in os.listdir(current_dir):
#     item_path = os.path.join(current_dir, item)
#     try:
#         if os.path.isfile(item_path) or os.path.islink(item_path):
#             os.remove(item_path)  # remove file or symlink
#         elif os.path.isdir(item_path):
#             shutil.rmtree(item_path)  # remove folder and its contents
#     except Exception as e:
#         print(f"Failed to delete {item_path}. Reason: {e}")

In [8]:
import torch
import torch.nn as nn
import numpy as np
import lightning as L
from lightning.pytorch import Trainer
from BLT.entropy import EntropyModel, EntropyConfig
from BLT.dataset import TokenDataset
from huggingface_hub import hf_hub_download

In [4]:
# download the tokenized text
hf_hub_download(repo_id="pt-sk/Text_Bytes_Tokens",
                filename="wikipedia_512_pretraining/tokenized_text1.npy",
                repo_type="dataset",
                local_dir="/kaggle/working/")

# load the tokenized text
tokens = np.load("/kaggle/working/wikipedia_512_pretraining/tokenized_text1.npy", allow_pickle=True)

wikipedia_512_pretraining/tokenized_text(…):   0%|          | 0.00/146M [00:00<?, ?B/s]

In [5]:
# Initialize model and config
config = EntropyConfig()
model = EntropyModel(config)

# count the number of parameters in the model
num_params = sum(p.numel() for p in model.parameters())
print(f"Number of parameters in the model: {num_params/1e6}M")

def configure_optimizer(model: nn.Module) -> torch.optim.Optimizer:
        # start with all of the candidate parameters (that require grad)
        param_dict = {pn: p for pn, p in model.named_parameters()}
        param_dict = {pn: p for pn, p in param_dict.items() if p.requires_grad}
        # create optim groups. Any parameters that is 2D will be weight decayed, otherwise no.
        # i.e. all weight tensors in matmuls + embeddings decay, all biases and layernorms don't.
        decay_params = [p for _, p in param_dict.items() if p.dim() >= 2]
        nodecay_params = [p for _, p in param_dict.items() if p.dim() < 2]

        # Create AdamW optimizer and use the fused version if available
        return torch.optim.AdamW([{'params': decay_params, 'weight_decay': 0.1},
                                {'params': nodecay_params, 'weight_decay': 0.0}],
                                lr=0.0001,
                                betas=(0.9, 0.999),
                                eps=1e-8,
                                fused=True)

optimizer = configure_optimizer(model)

Number of parameters in the model: 35.800064M


In [19]:
class GPTWrapper(L.LightningModule):
    def __init__(self, config: EntropyConfig, model: EntropyModel) -> None:
        super().__init__()
        self.config = config
        self.model = model
        self.optimizer = self.configure_optimizers()
    
    def training_step(self, batch, batch_idx):
        self.model.train()
        optimizer = self.optimizers()
        optimizer.zero_grad()
        
        inputs, targets = batch
        _, loss = self.model(inputs, targets)
        self.log("Train_Loss", loss, prog_bar=True)

        return loss
    
    def configure_optimizers(self):
        optimizer = configure_optimizer(self.model)
        return optimizer

In [20]:
dataset = TokenDataset(block_size=2048, input_ids=tokens)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=8)

model_wrapper = GPTWrapper(config, model)

In [21]:
trainer = Trainer(max_epochs=1,
                  accelerator="cuda",
                  devices=1)

INFO: 💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model_wrapper, dataloader)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name  | Type         | Params | Mode 
-----------------------------------------------
0 | model | EntropyModel | 35.8 M | train
-----------------------------------------------
35.8 M    Trainable params
0         Non-trainable params
35.8 M    Total params
143.200   Total estimated model params size (MB)
118       Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]